# Get Available Contracts with respective strike prices
## Chosen stock - BANKNIFTY

In [189]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options  
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time
import string
import shutil
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [74]:
## Configurations
download_location = r"/Users/mayank.gupta/Moneygen/Downloads"
data_file_location = r"/Users/mayank.gupta/Moneygen/DataFiles"
sleep_duration = 3
options = Options() 
options.add_experimental_option("prefs", {
  "download.default_directory": download_location,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})
# options.add_argument("--headless")  

### Load the chrome webdriver

In [75]:
driver = webdriver.Chrome(executable_path = './chromedriver', chrome_options = options)

/Users/mayank.gupta/venv/tradeai/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


### Get the web page using driver for BANKNIFTY

In [76]:
driver.get("https://nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuoteFO.jsp?underlying=BANKNIFTY&instrument=FUTIDX&type=-&strike=-&expiry=25JUL2019")
time.sleep(sleep_duration)

### Select 'Options' as instrument type

In [77]:
instrument_type_element = driver.find_element_by_id("instruments")
instrument_selector = Select(instrument_type_element)
## Index is always index + 1 because 0 index corresponds to 'Select...'
instrument_selector.select_by_index(2)
time.sleep(sleep_duration)

### Get expiry dates of all available contracts

In [78]:
expiry_dates_element = driver.find_element_by_id("expiryDates")
expiry_dates = expiry_dates_element.text.split('\n')[2:].copy()
time.sleep(sleep_duration)

In [79]:
expiry_dates

['25JUL2019',
 '18JUL2019',
 '11JUL2019',
 '04JUL2019',
 '01AUG2019',
 '08AUG2019',
 '29AUG2019',
 '14AUG2019',
 '22AUG2019',
 '26SEP2019']

### Select expiry dates and populate the strike price list

In [80]:
def getStrikePrices():
    date_price_dict = dict()
    expiry_dates_selector = Select(expiry_dates_element)
    for index in range(len(expiry_dates)):
        print('Getting strike prices for: ', expiry_dates[index], ', having index: ', index)
        indexer = index + 1
        date_price_dict[expiry_dates[index]] = dict()

        ## Select expiry for a contract
        expiry_dates_selector.select_by_index(indexer)
        time.sleep(sleep_duration)

        ## Selecting 'Call' option to populate 'Strike Price' list
        Select(driver.find_element_by_id("optionType")).select_by_index(1)
        time.sleep(sleep_duration)

        ## Populating strike prices for Call option of selected date
        strike_price_element = driver.find_element_by_id('strikePrices')
        date_price_dict[expiry_dates[index]]['Call'] = strike_price_element.text.split('\n')[2:].copy()
        print('Length of strike price list for ', expiry_dates[index], 'Call option: ', len(date_price_dict[expiry_dates[index]]['Call']))
        time.sleep(sleep_duration)

        ## Selecting 'Put' option to populate 'Strike Price' list
        Select(driver.find_element_by_id("optionType")).select_by_index(2)
        time.sleep(sleep_duration)

        ## Populating strike prices for Put option of selected date
        strike_price_element = driver.find_element_by_id('strikePrices')
        date_price_dict[expiry_dates[index]]['Put'] = strike_price_element.text.split('\n')[2:].copy()
        print('Length of strike price list for ', expiry_dates[index], 'Put option: ', len(date_price_dict[expiry_dates[index]]['Put']))
        time.sleep(sleep_duration)
    return date_price_dict

In [81]:
try:
    date_price_dict = getStrikePrices()
except:
    print('Exception occured, trying again')
    time.sleep(sleep_duration + 10)
    date_price_dict = getStrikePrices()

Getting strike prices for:  25JUL2019 , having index:  0
Length of strike price list for  25JUL2019 Call option:  64
Length of strike price list for  25JUL2019 Put option:  64
Getting strike prices for:  18JUL2019 , having index:  1
Length of strike price list for  18JUL2019 Call option:  61
Length of strike price list for  18JUL2019 Put option:  48
Getting strike prices for:  11JUL2019 , having index:  2
Length of strike price list for  11JUL2019 Call option:  64
Length of strike price list for  11JUL2019 Put option:  48
Getting strike prices for:  04JUL2019 , having index:  3
Length of strike price list for  04JUL2019 Call option:  64
Length of strike price list for  04JUL2019 Put option:  64
Getting strike prices for:  01AUG2019 , having index:  4
Length of strike price list for  01AUG2019 Call option:  29
Length of strike price list for  01AUG2019 Put option:  16
Getting strike prices for:  08AUG2019 , having index:  5
Length of strike price list for  08AUG2019 Call option:  36
Len

In [82]:
month_mapper = {
"JAN":"-01-",
"FEB":"-02-",
"MAR":"-03-",
"APR":"-04-",
"MAY":"-05-",
"JUN":"-06-",
"JUL":"-07-",
"AUG":"-08-",
"SEP":"-09-",
"OCT":"-10-",
"NOV":"-11-",
"DEC":"-12-"
}

In [83]:
expiry_dates_mapper = dict()
## Change month name to month number
for index, date in enumerate(expiry_dates):
    month_name = expiry_dates[index][2:5]
    month_number = month_mapper[month_name]
    expiry_dates_mapper[expiry_dates[index]] = expiry_dates[index].replace(month_name, month_number)

In [84]:
expiry_dates_mapper

{'25JUL2019': '25-07-2019',
 '18JUL2019': '18-07-2019',
 '11JUL2019': '11-07-2019',
 '04JUL2019': '04-07-2019',
 '01AUG2019': '01-08-2019',
 '08AUG2019': '08-08-2019',
 '29AUG2019': '29-08-2019',
 '14AUG2019': '14-08-2019',
 '22AUG2019': '22-08-2019',
 '26SEP2019': '26-09-2019'}

In [85]:
for old_key, new_key in zip(expiry_dates_mapper.keys(), expiry_dates_mapper.values()):
    date_price_dict[new_key] = date_price_dict.pop(old_key)

In [86]:
date_price_dict.keys()

dict_keys(['25-07-2019', '18-07-2019', '11-07-2019', '04-07-2019', '01-08-2019', '08-08-2019', '29-08-2019', '14-08-2019', '22-08-2019', '26-09-2019'])

# Get historical data for All contracts -> All Strike Prices

In [145]:
## Configurations
instrument_type = 'Index Options'
symbol = 'BANK NIFTY'
year = '2019'

In [146]:
# driver.close()
driver = webdriver.Chrome(executable_path = './chromedriver', chrome_options = options)
driver.get('https://www.nseindia.com/products/content/derivatives/equities/historical_fo.htm')

/Users/mayank.gupta/venv/tradeai/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [147]:
Select(driver.find_element_by_id('instrumentType')).select_by_index(3) ## Select `Index Options`
time.sleep(sleep_duration)
Select(driver.find_element_by_id('symbol')).select_by_index(4) ## Select `BANK NIFTY`
time.sleep(sleep_duration)
Select(driver.find_element_by_id('dateRange')).select_by_index(6) ## Select `BANK NIFTY`
time.sleep(sleep_duration)
Select(driver.find_element_by_id('year')).select_by_index(5) ## Select `BANK NIFTY`
time.sleep(sleep_duration)

In [148]:
expiry_date_element = driver.find_element_by_id('expiryDate') ## Get expiryDates on website
expiry_date_list = expiry_date_element.text.replace(' ','').split('\n') ## Remove white spaces and split by newline

In [136]:
## Skip dates 
skip_list = list()

In [137]:
## For Call options
for index, item in enumerate(expiry_date_list):
    print(item)
    if item in skip_list:
        print('Skipped item: ', item)
        continue
    if item in list(expiry_dates_mapper.values()):
        print('Inside if: ', item)
        Select(expiry_date_element).select_by_index(index)
        Select(driver.find_element_by_id("optionType")).select_by_index(1)
        time.sleep(sleep_duration)
        for strike_price in date_price_dict[item]['Call']:
            driver.find_element_by_id('strikePrice').clear()
            driver.find_element_by_id('strikePrice').send_keys(int(strike_price.split('.')[0]))
            driver.find_element_by_id('getButton').click()
            time.sleep(sleep_duration)
            try:
                download_link = WebDriverWait(driver, sleep_duration+20).until(EC.presence_of_element_located((By.LINK_TEXT, "Download file in csv format")))
                download_link.click()
            except:
                print('Exception occured, waiting and trying again')
                driver.find_element_by_id('getButton').click()
                download_link = WebDriverWait(driver, sleep_duration+20).until(EC.presence_of_element_located((By.LINK_TEXT, "Download file in csv format")))
                download_link.click()
            time.sleep(sleep_duration)
            try:
                filename = os.listdir(download_location)[0]
            except:
                print("Index error occured, waiting for a file to appear in directory")
                ## There might be a glitch due to which 'Download' link doesn't get clicked
                download_link.click()
                time.sleep(sleep_duration + 10)
                filename = os.listdir(download_location)[0]
            destination_filename = item + '_' + 'Call' + '_' + strike_price.split('.')[0] + '.csv'
            shutil.move(os.path.join(download_location,filename),os.path.join(data_file_location, destination_filename))
            skip_list.append(item)

SelectExpiry
03-01-2019
10-01-2019
17-01-2019
24-01-2019
31-01-2019
07-02-2019
14-02-2019
21-02-2019
28-02-2019
07-03-2019
14-03-2019
15-03-2019
20-03-2019
28-03-2019
04-04-2019
11-04-2019
18-04-2019
25-04-2019
02-05-2019
09-05-2019
16-05-2019
23-05-2019
30-05-2019
06-06-2019
13-06-2019
20-06-2019
21-06-2019
27-06-2019
04-07-2019
Skipped item:  04-07-2019
11-07-2019
Skipped item:  11-07-2019
18-07-2019
Skipped item:  18-07-2019
25-07-2019
Skipped item:  25-07-2019
01-08-2019
Skipped item:  01-08-2019
08-08-2019
Skipped item:  08-08-2019
14-08-2019
Skipped item:  14-08-2019
22-08-2019
Skipped item:  22-08-2019
29-08-2019
Skipped item:  29-08-2019
20-09-2019
26-09-2019
Inside if:  26-09-2019
26-12-2019


In [149]:
## skip_items
skip_list = list()

In [151]:
## For Put options
for index, item in enumerate(expiry_date_list):
    print(item)
    if item in skip_list:
        print('Skipped item: ', item)
        continue
    if item in list(expiry_dates_mapper.values()):
        print('Inside if: ', item)
        Select(expiry_date_element).select_by_index(index)
        Select(driver.find_element_by_id("optionType")).select_by_index(2)
        time.sleep(sleep_duration)
        for strike_price in date_price_dict[item]['Put']:
            driver.find_element_by_id('strikePrice').clear()
            driver.find_element_by_id('strikePrice').send_keys(int(strike_price.split('.')[0]))
            driver.find_element_by_id('getButton').click()
            time.sleep(sleep_duration)
            try:
                download_link = WebDriverWait(driver, sleep_duration+20).until(EC.presence_of_element_located((By.LINK_TEXT, "Download file in csv format")))
                download_link.click()
            except:
                print('Exception occured, waiting and trying again')
                driver.find_element_by_id('getButton').click()
                download_link = WebDriverWait(driver, sleep_duration+20).until(EC.presence_of_element_located((By.LINK_TEXT, "Download file in csv format")))
                download_link.click()
            time.sleep(sleep_duration)
            try:
                filename = os.listdir(download_location)[0]
            except:
                print("Index error occured, waiting for a file to appear in directory")
                ## There might be a glitch due to which 'Download' link doesn't get clicked
                download_link.click()
                time.sleep(sleep_duration + 10)
                filename = os.listdir(download_location)[0]
            destination_filename = item + '_' + 'Put' + '_' + strike_price.split('.')[0] + '.csv'
            shutil.move(os.path.join(download_location,filename),os.path.join(data_file_location, destination_filename))
            skip_list.append(item)
        

SelectExpiry
03-01-2019
10-01-2019
17-01-2019
24-01-2019
31-01-2019
07-02-2019
14-02-2019
21-02-2019
28-02-2019
07-03-2019
14-03-2019
15-03-2019
20-03-2019
28-03-2019
04-04-2019
11-04-2019
18-04-2019
25-04-2019
02-05-2019
09-05-2019
16-05-2019
23-05-2019
30-05-2019
06-06-2019
13-06-2019
20-06-2019
21-06-2019
27-06-2019
04-07-2019
Skipped item:  04-07-2019
11-07-2019
Inside if:  11-07-2019
18-07-2019
Inside if:  18-07-2019
25-07-2019
Inside if:  25-07-2019
Exception occured, waiting and trying again
01-08-2019
Inside if:  01-08-2019
08-08-2019
Inside if:  08-08-2019
14-08-2019
Inside if:  14-08-2019
22-08-2019
Inside if:  22-08-2019
29-08-2019
Inside if:  29-08-2019
20-09-2019
26-09-2019
Inside if:  26-09-2019
26-12-2019


# Append dataframe to respective Date-StrikePrice combo

In [ ]:
dataframe_dict = dict()

In [183]:
def process_data_files(data_directory: str, option_type: str, dataframe_dict: dict):
    
    files_path = os.path.join(data_directory, option_type)
    
    for file in os.listdir(files_path):
    #     print('Processing file:', file)
        filename_split = file.split('_')
        expiry_date = filename_split[0]
        strike_price = filename_split[2].split('.')[0]

        if expiry_date not in dataframe_dict.keys():
            dataframe_dict[expiry_date] = dict()

        if option_type not in dataframe_dict[expiry_date].keys():
            dataframe_dict[expiry_date][option_type] = dict()

        dataframe_dict[expiry_date][option_type][strike_price] = pd.read_csv(os.path.join(files_path, file))
        
    return dataframe_dict

In [184]:
dataframe_dict = process_data_files('DataFiles','Put',dict())

In [185]:
dataframe_dict = process_data_files('DataFiles','Call',dataframe_dict)

In [262]:
dataframe_dict_copy = dataframe_dict.copy()

## Experiments

In [244]:
ohlc = ['Open','High','Low','Close']

In [197]:
open_prices = np.array(dataframe_dict['11-07-2019']['Put']['31300']['Open'].tolist())

In [209]:
open_prices

array([  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  , 398.  , 481.7 , 455.65, 280.  ])

In [210]:
result = np.polyfit(range(0, len(open_prices)), open_prices, deg=3)
slope = result[-2]
# float(slope)
result

array([  0.09347727,  -2.95791143,  23.49089475, -32.88310973])

In [225]:
open_prices

array([  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  , 398.  , 481.7 , 455.65, 280.  ])

In [216]:
open_prices.shape[0]

30

In [218]:
blank = np.zeros(open_prices.shape[0])

In [230]:
for i in range(0,open_prices.shape[0]-1) : 
  
    # absolute difference between 
    # consecutive numbers 
    diff = open_prices[i+1] - open_prices[i]
    blank[i+1]=diff

In [231]:
blank

array([   0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,  398.  ,   83.7 ,
        -26.05, -175.65])

In [306]:
open_prices_df = dataframe_ohlc_all['04-07-2019']['Put']['28200'][ohlc]
open_prices_df = open_prices_df[open_prices_df['Open'] != 0]

In [307]:
open_prices_df

,Open,High,Low,Close
39,2,2.95,1.25,1.25
40,2,2.60,0.50,0.80
41,2,2.70,0.10,0.55


In [308]:
profit_loss_open = open_prices_df - open_prices_df.shift(1)

In [310]:
profit_loss_open['Open'].sum()

0.0

In [294]:
profit_loss_open

,Open,High,Low,Close
30,NaN,NaN,NaN,NaN
31,-76.20,29.85,-99.50,55.00
32,-63.80,0.15,4.50,-75.15
33,109.25,-60.75,-124.25,-186.45
34,-219.25,-125.15,-17.35,33.25
35,7.30,-49.85,3.35,-34.35
36,12.50,10.50,-81.75,-91.95
37,-124.80,-128.85,-72.30,-83.55
38,-97.00,-118.00,-33.70,-27.10
39,-5.35,7.55,3.10,-0.05


# Experiments done

## Remove dataframes with 0 in all ohlc data

In [258]:
def remove_strike_prices_with_only_zero_values(dataframe_dictionary: dict, option_type: str):
    dataframe_dict = dataframe_dictionary.copy()
    for expiry_date in list(dataframe_dict.keys()):
#         print("Expiry date:", expiry_date)
        for strike_price in list(dataframe_dict[expiry_date][option_type].keys()):
#             print("Strike Price:", strike_price)
            if dataframe_dict[expiry_date][option_type][strike_price]['Open'].sum() == 0:
                del dataframe_dict[expiry_date][option_type][strike_price]
    return dataframe_dict

In [263]:
dataframe_non_zero_put = remove_strike_prices_with_only_zero_values(dataframe_dict_copy, 'Put')
dataframe_non_zero_all = remove_strike_prices_with_only_zero_values(dataframe_non_zero_put, 'Call')

## Filter OHLC data in all dataframes

In [270]:
def filter_ohlc_data(dataframe_dictionary: dict, option_type: str):
    dataframe_dict = dataframe_dictionary.copy()
    for expiry_date in list(dataframe_dict.keys()):
#         print("Expiry date:", expiry_date)
        for strike_price in list(dataframe_dict[expiry_date][option_type].keys()):
#             print("Strike Price:", strike_price)
            dataframe_dict[expiry_date][option_type][strike_price] = dataframe_dict[expiry_date][option_type][strike_price][ohlc]
    return dataframe_dict

In [272]:
dataframe_ohlc_put = filter_ohlc_data(dataframe_non_zero_all, 'Put')
dataframe_ohlc_all = filter_ohlc_data(dataframe_ohlc_put, 'Call')

## Get profitable stocks from OHLC

In [297]:
## Criteria can be: Open, High, Low, Close
def list_profitable_stocks(dataframe_dictionary: dict, option_type: str, criteria: str): 
    dataframe_dict = dataframe_dictionary.copy()
    for expiry_date in list(dataframe_dict.keys()):
#         print("Expiry date:", expiry_date)
        for strike_price in list(dataframe_dict[expiry_date][option_type].keys()):
#             print("Strike Price:", strike_price)
            df = dataframe_dict[expiry_date][option_type][strike_price]
            profit_loss_df = df - df.shift(1)
            profit_loss_value = profit_loss_df[criteria].sum()
            if profit_loss_value > 0:
                print('Profit in:', expiry_date, option_type, strike_price, "with value: ", profit_loss_value)



In [298]:
dataframe_profitable_put = list_profitable_stocks(dataframe_ohlc_all, 'Put', 'Open')

Profit in: 11-07-2019 Put 31300 with value:  280.0
Profit in: 11-07-2019 Put 29000 with value:  11.1
Profit in: 11-07-2019 Put 30800 with value:  150.35
Profit in: 11-07-2019 Put 31100 with value:  247.65
Profit in: 11-07-2019 Put 31500 with value:  428.0
Profit in: 11-07-2019 Put 31700 with value:  565.75
Profit in: 11-07-2019 Put 30900 with value:  176.9
Profit in: 11-07-2019 Put 31000 with value:  210.65
Profit in: 11-07-2019 Put 31200 with value:  275.0
Profit in: 11-07-2019 Put 31600 with value:  508.5
Profit in: 11-07-2019 Put 31400 with value:  368.65
Profit in: 11-07-2019 Put 29500 with value:  16.1
Profit in: 11-07-2019 Put 30300 with value:  53.0
Profit in: 11-07-2019 Put 30700 with value:  132.7
Profit in: 11-07-2019 Put 30500 with value:  84.8
Profit in: 11-07-2019 Put 30200 with value:  50.0
Profit in: 11-07-2019 Put 30000 with value:  35.1
Profit in: 11-07-2019 Put 31900 with value:  650.0
Profit in: 11-07-2019 Put 32000 with value:  870.0
Profit in: 11-07-2019 Put 30400 

In [289]:
dataframe_profitable_call = list_profitable_stocks(dataframe_ohlc_all, 'Call', 'Open')

Profit in: 11-07-2019 Call 33000
Profit in: 11-07-2019 Call 31400
Profit in: 11-07-2019 Call 31600
Profit in: 11-07-2019 Call 31200
Profit in: 11-07-2019 Call 31000
Profit in: 11-07-2019 Call 30900
Profit in: 11-07-2019 Call 31700
Profit in: 11-07-2019 Call 33300
Profit in: 11-07-2019 Call 31500
Profit in: 11-07-2019 Call 31100
Profit in: 11-07-2019 Call 31300
Profit in: 11-07-2019 Call 32200
Profit in: 11-07-2019 Call 32000
Profit in: 11-07-2019 Call 31900
Profit in: 11-07-2019 Call 30000
Profit in: 11-07-2019 Call 32600
Profit in: 11-07-2019 Call 30500
Profit in: 11-07-2019 Call 32100
Profit in: 11-07-2019 Call 32300
Profit in: 11-07-2019 Call 30700
Profit in: 11-07-2019 Call 32500
Profit in: 11-07-2019 Call 31800
Profit in: 29-08-2019 Call 31500
Profit in: 29-08-2019 Call 33000
Profit in: 29-08-2019 Call 31000
Profit in: 29-08-2019 Call 32500
Profit in: 29-08-2019 Call 32000
Profit in: 04-07-2019 Call 30300
Profit in: 04-07-2019 Call 32700
Profit in: 04-07-2019 Call 31800
Profit in: